<h1 style="color:dodgerblue"> Deriving Some Summation Formul&aelig; with Python</h1>

In [169]:
from sympy import *

In [170]:
def find_poly_order(seq):
    """
    seq: is a finite sequence of integers generated by a polynomial
    This function returns the order of the polynomial which gives
    a the sume of the first n terms of the sequence. 
    """
    degree = 0;
    while sum(seq) != 0:
        degree += 1
        seq = [seq[i + 1] - seq[i] for i in range(len(seq) - 1)]
    # We should stop as soon as the sequence is constant, but we test for this by going calculating 
    # on to many differences, so we need to reduce our degree by 1
    return degree - 1 

In [1]:
def create_poly(degree):
    """
    degree: is and int
    """
    if degree > 30:
        raise ValueError('create_poly cannot generate polynomials with a degree greater than 30') 
    from sympy.parsing.sympy_parser import parse_expr
    from string import ascii_lowercase, ascii_uppercase
    # Get coefficient names 
    letters = (ascii_lowercase + ascii_uppercase).replace('x', '') # we can't have a coefficient of x
    coef = list(letters)[0:degree + 1]
    # Create a polynomial expression 
    pexp = ' + '.join(['(' + coef[i] + ' * x ** ' + str((len(coef) - i - 1)) + ')'
                       for i in range(len(coef) - 1)])
    pexp += ' + ' + coef[-1]
    # Return polynomial  
    return parse_expr(pexp)

In [4]:
def create_sys_of_eq(seq, poly):
    sys = []
    for i in range(Poly(poly).degree() + 1):
        sys.append(poly.subs(Symbol('x'), i) - seq[i]) 
    return sys

def solve_poly(seq, poly):
    sys = create_sys_of_eq(seq, poly)
    coeffs = Poly(poly, Symbol('x')).coeffs()
    sol = list(linsolve(sys, coeffs))[0]

    i = 0
    for c in Poly(poly, x).coeffs():
        poly = poly.subs(c, sol[i])
        i += 1
    return poly

In [23]:
def create_formula(f):
    # TODO: Determin the number of elements in the sequence necessary to solve the 
    # the formula. 
    seq = [sum([f(i) for i in range(1, n + 1)]) for n in range(1, 100)] 
    deg = find_poly_order(seq)
    general_poly = create_poly(deg)
    actual_poly = solve_poly(seq, general_poly)

    # We want our polynomial in terms of n, not x, as 
    # this how we traditionally represent the solution.
    # Also, we did all of our work assuming a zero index, but we actually 
    # want our sequence to start at one so we decrement n by one
    return factor(actual_poly.subs(Symbol('x'), Symbol('n') - 1))

<h2 style="color:dodgerblue">
$\sum_{i = 1}^n i$
</h2>

In [24]:
create_formula(lambda x: x)

n*(n + 1)/2

<h2 style="color:dodgerblue">
$\sum_{i = 1}^n i^2$
</h2>

In [7]:
create_formula(lambda x: x ** 2)

n*(n + 1)*(2*n + 1)/6

<h2 style="color:dodgerblue">
$\sum_{i = 1}^n i^3$
</h2>

In [8]:
create_formula(lambda x: x ** 3)

n**2*(n + 1)**2/4

<h2 style="color:dodgerblue">
$\sum_{i = 1}^n i^4$
</h2>

In [9]:
create_formula(lambda x: x ** 4)

n*(n + 1)*(2*n + 1)*(3*n**2 + 3*n - 1)/30

<h2 style="color:dodgerblue">
$\sum_{i = 1}^n i^5$
</h2>

In [10]:
create_formula(lambda x: x ** 5)

n**2*(n + 1)**2*(2*n**2 + 2*n - 1)/12

<h2 style="color:dodgerblue">
$\sum_{i = 1}^n i^6$
</h2>

In [11]:
create_formula(lambda x: x ** 6)

n*(n + 1)*(2*n + 1)*(3*n**4 + 6*n**3 - 3*n + 1)/42

<h2 style="color:dodgerblue">
$\sum_{i = 1}^n i^7$
</h2>

In [12]:
create_formula(lambda x: x ** 7)

n**2*(n + 1)**2*(3*n**4 + 6*n**3 - n**2 - 4*n + 2)/24

<h2 style="color:dodgerblue">
$\sum_{i = 1}^n i^8$
</h2>

In [13]:
create_formula(lambda x: x ** 8)

n*(n + 1)*(2*n + 1)*(5*n**6 + 15*n**5 + 5*n**4 - 15*n**3 - n**2 + 9*n - 3)/90

<h2 style="color:dodgerblue">
$\sum_{i = 1}^n i^9$
</h2>

In [14]:
create_formula(lambda x: x ** 9)

n**2*(n + 1)**2*(n**2 + n - 1)*(2*n**4 + 4*n**3 - n**2 - 3*n + 3)/20

<h2 style="color:dodgerblue">
$\sum_{i = 1}^n i^{10}$
</h2>

In [15]:
create_formula(lambda x: x ** 10)

n*(n + 1)*(2*n + 1)*(n**2 + n - 1)*(3*n**6 + 9*n**5 + 2*n**4 - 11*n**3 + 3*n**2 + 10*n - 5)/66

Conisder Implementing the Euler-Maclaurin method.

<h1 style="color:dodgerblue">Riemann Sums</h1>

In [171]:
from IPython.display import display, Math

In [188]:
def reimann_sum_latex(f, a, b, n, align='left'):
    from sympy.utilities.lambdify import implemented_function, lambdify
    dx = (S(b) - S(a)) / S(n)
    lb = 0 if align in ['left', 'mid'] else 1
    ub = n - 1 if align in ['left', 'mid'] else n
    x = Symbol('x')
    f = implemented_function('f', f)
    f = lambdify(x, f(x))
    i = Symbol('i')
    a = a if align != 'mid' else S(a) + dx / 2
    summand = latex(f(x).subs(x, a + i * dx))
    if '+' in summand or '-' in summand:
        if 'frac' in summand:
            summand = r'\bigg[' + summand + r'\bigg]'
        else:
            summand = r'\big[' + summand + r'\big]'
    formula = r'\sum_{{i = {}}}^{{{}}}{}\cdot{}'.format(lb, ub, summand,latex(dx))
    return formula

In [175]:
fml = riemann_sum_latex(lambda x: (x + 2) ** 3, 1, 5, 12, 'right')
display(Math(fml))

<IPython.core.display.Math object>

In [176]:
f = lambda x: sqrt(x) + 5
fml = riemann_sum_latex(f, 1, 3, 8, 'left')
display(Math(fml))

<IPython.core.display.Math object>

In [177]:
f = lambda x: (x - 1) ** 2
fml = riemann_sum_latex(f, -4, 4, 4, 'mid')
display(Math(fml))

<IPython.core.display.Math object>

In [180]:
f = lambda x: 1 / x + 2
fml = riemann_sum_latex(f, 2, 7, 10, 'right')
display(Math(fml))

<IPython.core.display.Math object>

In [185]:
f = lambda x: x ** 3
fml = riemann_sum_latex(f, 0, 2, 6, 'left')
display(Math(fml))

<IPython.core.display.Math object>

In [187]:
f = lambda x: tan(x)
fml = riemann_sum_latex(f, 0, pi/3, 6, 'mid')
display(Math(fml))

<IPython.core.display.Math object>

In [228]:
f = lambda x: -abs(x) + 7
fml = riemann_sum_latex(f, -1, 3, 4, 'right')
display(Math(fml))

<IPython.core.display.Math object>

In [204]:
round(5000/0.31, 0)

16129.0

In [205]:
print(f'{_ * 107:,.0f}')

1,725,803


In [227]:
print(f'{1.725e6 * .1/1000000:,.2f} million')

0.17 million


In [241]:
print('\N{Tiger Face}')

🐯


In [243]:
import ipywidgets as w

In [255]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [257]:
def f(x):
    return x

In [261]:
interact(f, x=10);

10

In [263]:
interact(f, x=10);

10

In [289]:
x = [f'my-var-{i}' for i in range(1, 11)]
x

['my-var-1',
 'my-var-2',
 'my-var-3',
 'my-var-4',
 'my-var-5',
 'my-var-6',
 'my-var-7',
 'my-var-8',
 'my-var-9',
 'my-var-10']

In [277]:
from functools import reduce

In [301]:
myvar = \
[''.join(m) for 
 m in [map(str.capitalize, l) for 
     l in [s.split('-') for s in x]]]
myvar

['MyVar1',
 'MyVar2',
 'MyVar3',
 'MyVar4',
 'MyVar5',
 'MyVar6',
 'MyVar7',
 'MyVar8',
 'MyVar9',
 'MyVar10']